In [16]:
import requests

from bs4 import BeautifulSoup

import pandas as pd

import csv

import os

from urllib.parse import urljoin

In [18]:
# Chemin du dossier "data"

data_folder = r'C:\Users\USER\Documents\PROJETS\BooksToScrapeSurveillancePrix\data'

# Créer un dossier pour stocker les images dans le dossier "data/images"

image_folder = os.path.join(data_folder, 'images')
if not os.path.exists(image_folder):
    os.makedirs(image_folder)

    
# Liste pour stocker les données des livres

books = []


In [22]:
# Boucle sur les pages du catalogue

for i in range(1,50):
     
     # Construire l'URL de la page du catalogue

     url = f"https://books.toscrape.com/catalogue/page-{i}.html"

     # Effectuer la requete HTTP

     response = requests.get(url)

     # Obtenir le contenu de la reponse

     response = response.content

     # Analyse le contenu HTML de la page

     soup = BeautifulSoup(response, 'html.parser')

     # Trouver la balise <ol> qui contient les articles

     ol = soup.find('ol')

     # Trouver tous les articles avec la classe 'product_prod'

     articles = ol.find_all('article', class_='product_pod')

     # Boucle sur chaque article

     for article in articles:
         
         # Trouver la balise 'img' pour obtenir l'URL de l'image
         image = article.find('img')
         image_url = image.attrs['src']

         # Utiliser urljoin pour construire l'URL complète
         image_url = urljoin(url, image_url)
 
       

         # Télécharger l'image avec gestion des exceptions
         try:
             response = requests.get(image_url, timeout=10)  # Ajouter un timeout si nécessaire
             response.raise_for_status()  # Vérifier si la requête a réussi
         except requests.exceptions.RequestException as e:
             print(f"Erreur lors du téléchargement de l'image: {e}")
             continue  # Passer à l'article suivant en cas d'erreur

    
         # Construire le chemin de sauvegarde local dans le dossier "data/images"
         image_name = image_url.split('/')[-1]
         image_path = os.path.join(image_folder, image_name)

         # Sauvegarder l'image localement
         with open(image_path, 'wb') as img_file:
          img_file.write(response.content)

         
         # Trouver la balise 'p' pour obtenir le titre

         image = article.find('img')
         title = image.attrs['alt']

         # Trouver la balise 'p' pour obtenir la classe de l'étoile

         star = article.find('p')
         star = star['class'][1]

         # Trouver la balise 'p' avec la classe 'price_color' pour obtenir le prix

         price = article.find('p', class_='price_color').text
         price = float(price[1:])

         # Ajouter les informations du livre à la liste
         books.append([title, price, star, image_path])


In [25]:
# Créer un DataFrame Pandas avec les données des livres

df = pd.DataFrame(books, columns=['title', 'Price', 'Star Rating', 'Image Product'])

# Sauvegarder le DataFrame dans un fichier Excel dans le dossier "data"

excel_path = os.path.join(data_folder, 'books.xlsx')
df.to_excel(excel_path, index=False)

In [26]:
# Afficher le contenu du 'Books.csv' 

df

,title,Price,Star Rating,Image Product
0,A Light in the Attic,51.77,Three,C:\Users\USER\Documents\PROJETS\BooksToScrapeS...
1,Tipping the Velvet,53.74,One,C:\Users\USER\Documents\PROJETS\BooksToScrapeS...
2,Soumission,50.10,One,C:\Users\USER\Documents\PROJETS\BooksToScrapeS...
3,Sharp Objects,47.82,Four,C:\Users\USER\Documents\PROJETS\BooksToScrapeS...
4,Sapiens: A Brief History of Humankind,54.23,Five,C:\Users\USER\Documents\PROJETS\BooksToScrapeS...
...,...,...,...,...
1051,Icing (Aces Hockey #2),40.44,Four,C:\Users\USER\Documents\PROJETS\BooksToScrapeS...
1052,"Hawkeye, Vol. 1: My Life as a Weapon (Hawkeye #1)",45.24,Three,C:\Users\USER\Documents\PROJETS\BooksToScrapeS...
1053,Having the Barbarian's Baby (Ice Planet Barbar...,34.96,Four,C:\Users\USER\Documents\PROJETS\BooksToScrapeS...
1054,"Giant Days, Vol. 1 (Giant Days #1-4)",56.76,Four,C:\Users\USER\Documents\PROJETS\BooksToScrapeS...
